### Import packages

In [60]:
using Oceananigans #use v.1.10
using Oceananigans.Units
using Oceananigans.OutputReaders: FieldTimeSeries
using Oceananigans.BoundaryConditions
using Oceananigans.TurbulenceClosures
#using CairoMakie 
using NCDatasets

## Define Model Parameters

In [61]:
# Model parameters
Nx = 300
Nz = 80
f = 1e-4               # Coriolis frequency [s⁻¹]
L_front = 10kilometers  # Initial front width [m]
aspect_ratio = 100      # L/H
δ = 0                   # Strain ratio (α/f)
Ro = 1/2             # Rossby number (defines M^2)
F = Inf              # Froude number (N² = M²/(F²H))
Re_h = 1e10    # horizontal reynolds number
Re_v = +Inf              #vertical reynolds number
n = 2              #diffusivity number                 

sponge_width = 8kilometers
damping_rate = f

# Derived parameters
H_front = L_front/aspect_ratio
α = f*δ
M² = (Ro^2*f^2*L_front)/H_front
N² = (M²*L_front)/(F^2*H_front)
Bu = Ro/F
Δb = M²*L_front
κh = (sqrt(Δb*H_front)*L_front^(n-1))/Re_h
κv = κh*(Re_h/Re_v)*(H_front/L_front)^n




filename = "ST13_δ="*string(δ)*"_Ro="*string(Ro)*"_F="*string(F)*"_Re_h="*string(Re_h)
filename = "multiple_waves4"

println("Filename: ", filename)
println("\nDerived parameters:")
println("H = ", H_front/1000, " km")
println("α = ", α, " s⁻¹")
println("M² = ", M², " s⁻²")
println("N² = ", N², " s⁻²")
println("Bu = ", Bu)
println("κh = ", κh)
println("κv = ", κv)

Filename: multiple_waves4

Derived parameters:
H = 0.1 km
α = 0.0 s⁻¹
M² = 2.5e-7 s⁻²
N² = 0.0 s⁻²
Bu = 0.0
κh = 5.0e-7
κv = 0.0


## Define Model Domain

In [62]:

Lx = 64*L_front #40kilometers
Lz = H_front

grid = RectilinearGrid(size = (Nx, Nz), #in ST15 they use Nx = 200*L_front, Nz = 100*H_front
                       x = (-Lx/2, Lx/2),
                       z = (-Lz, 0),
                       topology = (Bounded, Flat, Bounded))

300×1×80 RectilinearGrid{Float64, Bounded, Flat, Bounded} on CPU with 3×0×3 halo
├── Bounded  x ∈ [-320000.0, 320000.0] regularly spaced with Δx=2133.33
├── Flat y                             
└── Bounded  z ∈ [-100.0, 0.0]         regularly spaced with Δz=1.25

## Forcing Terms

In [63]:
# advective forcing term 
u_background = XFaceField(grid)
u_background = - α * xnodes(grid, Face(), Center(), Center())
background_flow = AdvectiveForcing(u = u_background)

# no addtional u forcing





# v forcing
v_forcing_func(x, z, t, v, α) = - 2*α*v
v_forcing = Forcing(v_forcing_func, parameters = α, field_dependencies = :v )

# w forcing
w_forcing_func(x, z, t, w, α) = - α*w
w_forcing = Forcing(w_forcing_func, parameters = α, field_dependencies = :w )

# b forcing
b_forcing_func(x, z, t, α, b ) = - α*b
b_forcing = Forcing(b_forcing_func, parameters = α, field_dependencies= :b )



ContinuousForcing{Float64}
├── func: b_forcing_func (generic function with 1 method)
├── parameters: 0.0
└── field dependencies: (:b,)

## Sponge Layer

In [64]:
Δb = L_front * M²

target_buoyancy_left(x,z,t) = z*N²
target_buoyancy_right(x,z,t) = z*N² + Δb
    




left_mask_3D   = GaussianMask{:x}(center=-grid.Lx/2, width = sponge_width)
left_mask(x,z) = left_mask_3D(x,0,z)
uvw_sponge_left = Relaxation(rate=damping_rate, mask=left_mask)
b_sponge_left = Relaxation(rate=damping_rate, mask=left_mask, target = target_buoyancy_left) 

right_mask_3D  = GaussianMask{:x}(center=grid.Lx/2,width = sponge_width)
right_mask(x,z) = right_mask_3D(x,0,z)
uvw_sponge_right = Relaxation(rate=damping_rate, mask=right_mask)
b_sponge_right = Relaxation(rate=damping_rate, mask=right_mask, target = target_buoyancy_right ) 



Relaxation{Float64, typeof(right_mask), typeof(target_buoyancy_right)}
├── rate: 0.0001
├── mask: right_mask (generic function with 1 method)
└── target: target_buoyancy_right (generic function with 1 method)

## Boundary conditions

In [65]:

using Oceananigans.BoundaryConditions

# Free-slip for u and v (∂u/∂z = ∂v/∂z = 0)
free_slip = FieldBoundaryConditions(
    top = GradientBoundaryCondition(0.0),
    bottom = GradientBoundaryCondition(0.0)
)

# No vertical flow (w = 0 at top/bottom)
no_penetration = FieldBoundaryConditions(
    top = ValueBoundaryCondition(0.0),
    bottom = ValueBoundaryCondition(0.0)
)

velocity_bcs = (
    u = free_slip,
    v = free_slip,
    w = no_penetration
)



(u = Oceananigans.FieldBoundaryConditions, with boundary conditions
├── west: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── east: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── south: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── north: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── bottom: GradientBoundaryCondition: 0.0
├── top: GradientBoundaryCondition: 0.0
└── immersed: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing), v = Oceananigans.FieldBoundaryConditions, with boundary conditions
├── west: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── east: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── south: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── north: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── bottom: GradientBoundaryCondition: 0.0
├── top: GradientBoundaryCondition: 0.0
└── immersed: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing), w = O

## Buoyancy background conditions

In [66]:

∂b∂z_top = N²
∂b∂z_bottom = N²

buoyancy_bcs = FieldBoundaryConditions(
    top = GradientBoundaryCondition(∂b∂z_top),
    bottom = GradientBoundaryCondition(∂b∂z_bottom)
)
    

Oceananigans.FieldBoundaryConditions, with boundary conditions
├── west: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── east: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── south: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── north: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── bottom: GradientBoundaryCondition: 0.0
├── top: GradientBoundaryCondition: 0.0
└── immersed: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)

## Closure

In [67]:
horizontal_closure = HorizontalScalarDiffusivity(ν=κh, κ=κh )
vertical_closure = VerticalScalarDiffusivity(ν=κv , κ=κv )
closure = (horizontal_closure, vertical_closure)

(HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=5.0e-7, κ=5.0e-7), VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0, κ=0.0))

## Defining The Model

In [68]:
#with sponge

#=
model = NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = :b,
                advection = WENO(),
                forcing = (; u = (background_flow, uvw_sponge_left, uvw_sponge_right),
                             v = (background_flow , v_forcing, uvw_sponge_left, uvw_sponge_right) , 
                             w = (background_flow , w_forcing, uvw_sponge_left, uvw_sponge_right),
                             b = (background_flow, b_forcing, b_sponge_left, b_sponge_right)),
                boundary_conditions = (; b=buoyancy_bcs, velocity_bcs),
                closure = closure

                )
=#

#without sponge 

model = NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = :b,
                advection = WENO(),
                forcing = (; u = (background_flow),
                             v = (background_flow , v_forcing) , 
                             w = (background_flow , w_forcing),
                             b = (background_flow, b_forcing))
                #boundary_conditions = (; b=buoyancy_bcs, velocity_bcs)
                )





NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 300×1×80 RectilinearGrid{Float64, Bounded, Flat, Bounded} on CPU with 3×0×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO{3, Float64, Float32}(order=5)
├── tracers: b
├── closure: Nothing
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.0001)

## Initial Conditions

In [69]:
#inital setup

Δb = L_front * M²  # buoyancy jump across front  

#bᵢ(x, z) = N² * z + Δb * 1/2* (tanh(x/L_front) + 1) #for ST13 setup

#thermal_wind_balance(x,z) = (1/2 * N² * z^2 + Δb * 1/2* z * (tanh(x/L_front) + 1))/f

bᵢ(x, z) = N² * z + Δb * 1/2* (sin(x/L_front) + 1)  #for B00 setup


set!(model, b= bᵢ, u = 0, v = 0, w = 0)  # Start from rest


## Define Simulation

In [70]:
simulation = Simulation(model, Δt=20minutes, stop_time=30days)

Simulation of NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 20 minutes
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 30 days
├── Stop iteration: Inf
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => 4
│   ├── stop_iteration_exceeded => -
│   ├── wall_time_limit_exceeded => e
│   └── nan_checker => }
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [71]:
conjure_time_step_wizard!(simulation, IterationInterval(20), cfl=0.2, max_Δt=20minutes)

In [72]:
using Printf

wall_clock = Ref(time_ns())

function print_progress(sim)
    u, v, w = model.velocities
    progress = 100 * (time(sim) / sim.stop_time)
    elapsed = (time_ns() - wall_clock[]) / 1e9

    @printf("[%05.2f%%] i: %d, t: %s, wall time: %s, max(u): (%6.3e, %6.3e, %6.3e) m/s, next Δt: %s\n",
            progress, iteration(sim), prettytime(sim), prettytime(elapsed),
            maximum(abs, u), maximum(abs, v), maximum(abs, w), prettytime(sim.Δt))

    wall_clock[] = time_ns()

    return nothing
end

add_callback!(simulation, print_progress, IterationInterval(100))

## Diagnostic/Output

In [73]:
# Output setup
g = 9.81
u, v, w = model.velocities
ζ = ∂z(u) - ∂x(w)  # Vorticity in x-z plane
b = model.tracers.b

# Compute Ri
Ri = N² / (∂z(u)^2 + ∂z(v)^2) 
M_squared = ∂x(b)

#X = x + v/f
dbdX = ∂x(b)*(1/(1 + (1/f) * ∂x(v)))




#=
#For Julia animation
simulation.output_writers[:fields] = JLD2Writer(
    model, (; b, ζ , u, v, w),
    filename=filename * ".jld2",
    schedule=TimeInterval(0.5day),
    overwrite_existing=true
    )
=#
#=

#For Lagrangian filtering
simulation.output_writers[:fields] = JLD2Writer(
    model, (; b, u, v, w, Ri, M_squared, dbdX), filename=filename * ".jld2", schedule=TimeInterval(10minutes), overwrite_existing=true)
=#


#for python visualisation
simulation.output_writers[:fields] = NetCDFWriter(
    model, (; b, u, v, w, Ri, M_squared, dbdX), filename=filename * ".nc", schedule=TimeInterval(10minutes), overwrite_existing=true)

NetCDFWriter scheduled on TimeInterval(10 minutes):
├── filepath: multiple_waves4.nc
├── dimensions: time(0), x_faa(301), x_caa(300), z_aaf(81), z_aac(80)
├── 7 outputs: (v, w, dbdX, Ri, b, u, M_squared)
└── array type: Array{Float32}
├── file_splitting: NoFileSplitting
└── file size: 28.1 KiB

## Run Simulation

In [74]:
@info "Running the simulation..."

run!(simulation)

@info "Simulation completed in " * prettytime(simulation.run_wall_time)

[ Info: Running the simulation...
[ Info: Initializing simulation...


[00.00%] i: 0, t: 0 seconds, wall time: 13.902 seconds, max(u): (0.000e+00, 0.000e+00, 0.000e+00) m/s, next Δt: 20 minutes


[ Info:     ... simulation initialization complete (10.010 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (3.446 seconds).


[01.57%] i: 100, t: 11.333 hours, wall time: 35.546 seconds, max(u): (5.129e-02, 1.232e-01, 4.069e-04) m/s, next Δt: 8.965 minutes
[02.73%] i: 200, t: 19.667 hours, wall time: 31.603 seconds, max(u): (3.670e-02, 3.314e-02, 2.025e-04) m/s, next Δt: 7.066 minutes
[03.89%] i: 300, t: 1.167 days, wall time: 31.942 seconds, max(u): (2.114e-02, 1.371e-01, 2.333e-04) m/s, next Δt: 8.612 minutes
[05.05%] i: 400, t: 1.514 days, wall time: 29.920 seconds, max(u): (1.849e-02, 4.794e-02, 1.464e-04) m/s, next Δt: 7.052 minutes
[06.20%] i: 500, t: 1.861 days, wall time: 36.381 seconds, max(u): (3.040e-02, 1.330e-01, 1.746e-04) m/s, next Δt: 7.558 minutes
[07.36%] i: 600, t: 2.208 days, wall time: 32.490 seconds, max(u): (3.550e-02, 4.701e-02, 2.835e-04) m/s, next Δt: 6.665 minutes
[08.52%] i: 700, t: 2.556 days, wall time: 32.746 seconds, max(u): (4.191e-02, 1.379e-01, 2.938e-04) m/s, next Δt: 7.619 minutes
[09.68%] i: 800, t: 2.903 days, wall time: 32.040 seconds, max(u): (4.963e-02, 4.423e-02, 4.6

LoadError: NetCDF error: [31mOpening path c:\Users\Tom Cummings\Documents\Oceananigans_projects\multiple_waves4.nc: NetCDF: HDF error[39m (NetCDF error code: -101)

## Oceananigans Animation
(Won't work if you comment out julia animation field writer )

In [ ]:
# Visualization
b_ts = FieldTimeSeries(filename * ".jld2", "b")
ζ_ts = FieldTimeSeries(filename * ".jld2", "ζ")

u_ts = FieldTimeSeries(filename * ".jld2", "u")
v_ts = FieldTimeSeries(filename * ".jld2", "v")
w_ts = FieldTimeSeries(filename * ".jld2", "w")

times = b_ts.times


LoadError: BoundsError: attempt to access 0-element Vector{String} at index [1]

In [ ]:
# Coordinates
x = xnodes(grid, Center())# ./ 1e2  # km
z = znodes(grid, Center())# ./ 1e2  # km

In [ ]:
#using CairoMakie
fig = Figure(size=(1800, 1000))

In [ ]:
# Animation setup
n = Observable(1)
b_slice = @lift interior(b_ts[$n], :, 1, :)
ζ_slice = @lift interior(ζ_ts[$n], :, 1, :)

u_slice = @lift interior(u_ts[$n], :, 1, :)
v_slice = @lift interior(v_ts[$n], :, 1, :)
w_slice = @lift interior(w_ts[$n], :, 1, :)

title_text = @lift "Day $(round(times[$n]/day, digits=1))"

In [ ]:


# Animations of Buoyancy and Vorticity
empty!(fig)

# Buoyancy plot
ax_b = Axis(fig[1, 1], title="Buoyancy", xlabel="y [m]", ylabel="z [m]")
hm_b = heatmap!(ax_b, x, z, b_slice, colorrange=(0, Δb), colormap=:thermal)
Colorbar(fig[1, 2], hm_b, label="Buoyancy [m s⁻²]")
#(ax_b, x, z, b_slice ; levels=10, color=:black, linewidth=0.5)


# Vorticity plot
ax_ζ = Axis(fig[1, 3], title="Vorticity", xlabel="y [m]")
hm_ζ = heatmap!(ax_ζ, x, z, ζ_slice, colorrange=(-3e-3, 3e-3), colormap=:balance)
Colorbar(fig[1, 4], hm_ζ, label="Vorticity [s⁻¹]")
contour!(ax_ζ, x, z, b_slice ; levels=10, color=:black, linewidth=0.5)

Label(fig[0, :], title_text, fontsize=24)

# Create animation
frames = 1:length(times)
record(fig, filename * "_julia_animation.mp4", frames, framerate=8) do i
    n[] = i
end

fig  # Display final frame 


     



In [ ]:
#=

#animation of U only


empty!(fig)

ax_u = Axis(fig[1, 1], title="U", xlabel="y [m]", ylabel="z [m]")
hm = heatmap!(ax_u, x, z, u_slice; colorrange=(-5e-5, 5e-5), colormap=:balance)
Colorbar(fig[1, 2], hm_b, label="V [m s⁻1]")
contour!(ax_u, x, z, b_slice, levels=10, color=:black, linewidth=0.5)


# Create animation
frames = 1:length(times)
record(fig, filename * "_animation_velocities.mp4", frames, framerate=8) do i
    n[] = i
end

fig  # Display final frame

=#

In [ ]:
#animation of W only

#=

empty!(fig)

ax_w = Axis(fig[1, 1], title="W", xlabel="y [m]", ylabel="z [m]")
hm = heatmap!(ax_w, x, z, w_slice; colorrange=(-5e-5, 5e-5), colormap=:balance)
Colorbar(fig[1, 2], hm_b, label="W [m s⁻1]")
#contour!(ax_u, x, z, b_slice, levels=10, color=:black, linewidth=0.5)


# Create animation
frames = 1:length(times)
record(fig, filename * "_animation_velocities.mp4", frames, framerate=8) do i
    n[] = i
end

fig  # Display final frame

=#